# ETL
This notebook is used to extract and transform the data from the CSV files in `/Datasets` and load them into the SQLite database.

In [1]:
# Dependencies
import pandas as pd
from pathlib import Path
import requests

# Import SQL Alchemy
from sqlalchemy import create_engine

# Import and establish Base for which classes will be constructed 
import sqlalchemy
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func
from sqlalchemy import desc

# Import modules to declare columns and column data types
from sqlalchemy import Column, Integer, String, Float, Boolean

# Local modules
from config import omdb_api_key

## Import CSVs
#### Movies CSV
- Source: `Datasets/movies.csv`
- Table: `movies` in `movies_db.sqlite`
#### Omdb CSV
- Source: `Datasets/omdb.csv`
- Table: `omdb` in `movies_db.sqlite`
#### Actor CSV
- Source: `Datasets/actor.csv`
- Table: `actor` in `movies_db.sqlite`
#### Character CSV
- Source: `Datasets/character.csv`
- Table: `character` in `movies_db.sqlite`

In [2]:
# Get the movies data
movies_csv = Path('../Datasets/movies.csv')
movies_df = pd.read_csv(movies_csv)

# Get the datatype from the DataFrame
movies_df.dtypes

movieid           int64
title            object
mpaa_rating      object
budget            int64
gross             int64
release_date     object
genre            object
runtime           int64
rating          float64
rating_count    float64
summary          object
dtype: object

In [3]:
# Get the omdb data
omdb_csv = Path('../Datasets/omdb.csv')
omdb_movies_df = pd.read_csv(omdb_csv)

# Get the datatype from the DataFrame
omdb_movies_df.dtypes

Title            object
Year             object
Rated            object
Released         object
Runtime          object
Genre            object
Director         object
Writer           object
Actors           object
Plot             object
Language         object
Country          object
Awards           object
Poster           object
Ratings          object
Metascore       float64
imdbRating      float64
imdbVotes        object
imdbID           object
Type             object
DVD              object
BoxOffice        object
Production       object
Website         float64
Response           bool
Error            object
totalSeasons    float64
dtype: object

In [4]:
# Removing rows with no data or irrelevant data
omdb_movies_df.drop(omdb_movies_df.loc[omdb_movies_df['Error'] == 'Movie not found!'].index, inplace=True)
omdb_movies_df.drop(omdb_movies_df.loc[omdb_movies_df['Type'] == 'series'].index, inplace=True)

In [5]:
# Get the actors data
actor_df = Path('../Datasets/actor.csv')
actor_df = pd.read_csv(actor_df)

# Get the datatype from the DataFrame
actor_df.dtypes

actorid            int64
name              object
date_of_birth     object
birth_city        object
birth_country     object
height_inches    float64
biography         object
gender            object
ethnicity         object
networth         float64
dtype: object

In [6]:
# Get the character data
characters_df = Path('../Datasets/character.csv')
characters_df = pd.read_csv(characters_df)

# Get the datatype from the DataFrame
characters_df.dtypes

movieid             int64
actorid             int64
character_name     object
creditorder         int64
pay               float64
screentime         object
dtype: object

In [7]:
# Get Base
Base = declarative_base()

In [8]:
# Create movies class
class Movies(Base):
    __tablename__ = 'movies'
    movieid = Column(Integer, primary_key=True)
    title = Column(String)
    mpaa_rating = Column(String)
    budget = Column(String)
    gross = Column(String)
    release_date = Column(String)
    genre = Column(String)
    runtime = Column(Integer)
    rating = Column(Float)
    rating_count = Column(Float)
    summary = Column(String)     

In [9]:
# Create omdb class
class Omdb(Base):
    __tablename__ = 'omdb'
    id = Column(Integer, primary_key=True)
    Title = Column(String)
    Year = Column(Integer)
    Rated = Column(String)
    Released = Column(String)
    Runtime = Column(String)
    Genre = Column(String)
    Director = Column(String)   
    Writer = Column(String)
    Actors = Column(String)
    Plot = Column(String)
    Language = Column(String)
    Country = Column(String)
    Awards = Column(String)
    Poster = Column(String)
    Ratings = Column(String)
    Metascore = Column(Float)
    imdbRating = Column(Float)
    imdbVotes = Column(String)
    imdbID = Column(String)
    Type = Column(String)
    DVD = Column(String)
    BoxOffice = Column(String)
    Production = Column(String)

In [10]:
# Create actors class
class Actor(Base):
    __tablename__ = 'actor'
    actorid = Column(Integer, primary_key=True)
    name = Column(String)
    date_of_birth = Column(String)
    birth_city = Column(String)
    birth_country = Column(String)
    height_inches = Column(Float)
    biography = Column(String) 
    gender = Column(String)
    ethnicity = Column(String)
    networth = Column(Float)

In [11]:
# Create character class
class Character(Base):
    __tablename__ = 'character'
    id = Column(Integer, primary_key=True)
    movieid = Column(Integer)
    actorid = Column(Integer)
    character_name = Column(String)

In [12]:
# Create a connection to a SQLite database
engine = create_engine('sqlite:///movies_db.sqlite')

# Create the tables within the database
Base.metadata.create_all(engine)
# Start session
session = Session(bind=engine)

In [13]:
# Loop through movies DataFrame and retrieve data
for index, row in movies_df.iterrows():
    movieid = int(movies_df.loc[index,'movieid'])
    title = movies_df.loc[index,'title']
    mpaa_rating = movies_df.loc[index,'mpaa_rating']
    budget = movies_df.loc[index,'budget']
    gross = movies_df.loc[index,'gross']
    release_date = movies_df.loc[index,'release_date']
    genre = movies_df.loc[index,'genre']
    runtime = movies_df.loc[index,'runtime']
    rating = movies_df.loc[index,'rating']
    rating_count = movies_df.loc[index,'rating_count']
    summary = movies_df.loc[index,'summary']

    # Add data to database
    session.add(Movies(
        movieid = movieid,
        title = title,
        mpaa_rating = mpaa_rating,
        budget = budget,
        gross = gross,
        release_date = release_date,
        genre = genre,
        runtime = runtime,
        rating = rating,
        rating_count = rating_count,
        summary = summary
    ))

print(f"{len(movies_df)} rows ready for commit.")


636 rows ready for commit.


In [14]:
# Loop through omdb DataFrame and retrieve data
for index, row in omdb_movies_df.iterrows():   
    id = index
    Title = omdb_movies_df.loc[index,'Title']
    Year = int(omdb_movies_df.loc[index,'Year'])
    Rated = omdb_movies_df.loc[index,'Rated']
    Released = omdb_movies_df.loc[index,'Released']
    Runtime = omdb_movies_df.loc[index,'Runtime']
    Genre = omdb_movies_df.loc[index,'Genre']
    Director = omdb_movies_df.loc[index,'Director']
    Writer = omdb_movies_df.loc[index,'Writer']
    Actors = omdb_movies_df.loc[index,'Actors']
    Plot = omdb_movies_df.loc[index,'Plot']
    Language = omdb_movies_df.loc[index,'Language']
    Country = omdb_movies_df.loc[index,'Country']
    Awards = omdb_movies_df.loc[index,'Awards']
    Poster = omdb_movies_df.loc[index,'Poster']
    Ratings = omdb_movies_df.loc[index,'Ratings']
    Metascore = omdb_movies_df.loc[index,'Metascore']
    imdbRating = omdb_movies_df.loc[index,'imdbRating']
    imdbVotes = omdb_movies_df.loc[index,'imdbVotes']
    imdbID = omdb_movies_df.loc[index,'imdbID']
    Type = omdb_movies_df.loc[index,'Type']
    DVD = omdb_movies_df.loc[index,'DVD']
    BoxOffice = omdb_movies_df.loc[index,'BoxOffice']
    Production = omdb_movies_df.loc[index,'Production']


    # Add data to database
    session.add(Omdb(
        id = id,
        Title =  Title,
        Year =   Year,
        Rated =   Rated,
        Released =  Released,
        Runtime =  Runtime,
        Genre =   Genre,
        Director =  Director,
        Writer =   Writer,
        Actors =   Actors,
        Plot =   Plot,
        Language =  Language,
        Country =   Country,
        Awards =   Awards,
        Poster =   Poster,
        Ratings = Ratings,
        Metascore =  Metascore,
        imdbRating =  imdbRating,
        imdbVotes = imdbVotes,
        imdbID = imdbID,
        Type = Type,
        DVD =   DVD,
        BoxOffice =  BoxOffice,
        Production = Production
    ))
print(f"{len(omdb_movies_df)} rows ready for commit.")

624 rows ready for commit.


In [15]:
# Loop through actor DataFrame and retrieve data
for index, row in actor_df.iterrows():
    actorid = int(actor_df.loc[index,'actorid'])
    name = actor_df.loc[index,'name']
    date_of_birth = actor_df.loc[index,'date_of_birth']
    birth_city = actor_df.loc[index,'birth_city']
    birth_country = actor_df.loc[index,'birth_country']
    height_inches = actor_df.loc[index,'height_inches']
    biography = actor_df.loc[index,'biography']
    gender = actor_df.loc[index,'gender']
    ethnicity = actor_df.loc[index,'ethnicity']
    networth = actor_df.loc[index,'networth']
    
    # Add data to database
    session.add(Actor(
        actorid = actorid,
        name = name,
        date_of_birth = date_of_birth,
        birth_city = birth_city,
        birth_country = birth_country,
        height_inches = height_inches,
        biography = biography,
        gender = gender,
        ethnicity = ethnicity,
        networth = networth
    ))
  
print(f"{len(actor_df)} rows ready for commit.")

2599 rows ready for commit.


In [16]:
# Loop through characters DataFrame and retrieve data
for index, row in characters_df.iterrows():
    id = index
    movieid = int(characters_df.loc[index,'movieid'])
    actorid = int(characters_df.loc[index,'actorid'])
    character_name = characters_df.loc[index,'character_name']

    # Add data to database
    session.add(Character(
    id = id,
    movieid = movieid,
    actorid = actorid,
    character_name = character_name

    ))
print(f"{len(characters_df)} rows ready for commit.")

4319 rows ready for commit.


In [17]:
# Commit changes to session
session.commit()

# Close session
session.close()